In [1]:
# Libraries

import geemap, ee
import matplotlib.pyplot as plt

print("Libraries imported")

# Authenticate Google Earth Engine and initialize project

ee.Authenticate()
ee.Initialize(project="geog-581-483717")

print("Authenticated and initialized")

/Users/marcel/Desktop/geog581-gee-notebook/venv/lib/python3.10/site-packages/google/api_core/_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.4) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


Libraries imported
Authenticated and initialized


In [2]:
# Load Sentinel-2 image collection

sentinel_2 = "COPERNICUS/S2_HARMONIZED"

try:
    S2 = ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
except:
    print(f"Failed to load dataset {sentinel_2}")
else:
    print(f"Successfully loaded {sentinel_2}")

Successfully loaded COPERNICUS/S2_HARMONIZED


In [3]:
leconte = ee.Geometry.Point([-132.3513889, 56.8380556])
image = S2.filterDate('2024-03-01', '2024-07-31').filterBounds(leconte).sort('CLOUD_COVER').first()

In [9]:
image_info = image.getInfo()
dt = image_info["bands"][0]["data_type"]
image_min = dt["min"]
image_max = dt["max"]

image_minmax = image.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=image.geometry(),
    scale=10,
    bestEffort=True,
    maxPixels=1e9,
    tileScale=4,
)

minmax_info = image_minmax.getInfo()
print(minmax_info)


image_min_actual = ee.Number(image_minmax.get("B4_min"))
image_max_actual = ee.Number(image_minmax.get("B4_max"))
image_range = image_max - image_min
denom = (image_max_actual.subtract(image_min_actual))
image_stretched = (image.subtract(image_min_actual).divide(denom).multiply(image_range))


{'B10_max': 4273, 'B10_min': 0, 'B11_max': 8374, 'B11_min': 0, 'B12_max': 8409, 'B12_min': 0, 'B1_max': 18304, 'B1_min': 0, 'B2_max': 21533, 'B2_min': 0, 'B3_max': 20284, 'B3_min': 0, 'B4_max': 24311, 'B4_min': 0, 'B5_max': 24871, 'B5_min': 0, 'B6_max': 25220, 'B6_min': 0, 'B7_max': 24871, 'B7_min': 0, 'B8A_max': 24699, 'B8A_min': 0, 'B8_max': 25906, 'B8_min': 0, 'B9_max': 18459, 'B9_min': 0, 'MSK_CLASSI_CIRRUS_max': 1, 'MSK_CLASSI_CIRRUS_min': 0, 'MSK_CLASSI_OPAQUE_max': 1, 'MSK_CLASSI_OPAQUE_min': 0, 'MSK_CLASSI_SNOW_ICE_max': 1, 'MSK_CLASSI_SNOW_ICE_min': 0, 'QA10_max': None, 'QA10_min': None, 'QA20_max': None, 'QA20_min': None, 'QA60_max': 2048, 'QA60_min': 0}


In [11]:
rgb_vis = {
    "min": 0,
    "max": 20000,
    "bands": ['B4', 'B3', 'B2']}

# Initialize map container
Map = geemap.Map(ee_initialize=False)

# Add all four layers to map container
Map.addLayer(image_stretched, rgb_vis, 'LeConte RGB')

# Center around point of interest
Map.centerObject(ee.Geometry(leconte), 8)

# Show map
Map

Map(center=[56.838055600000004, -132.3513889], controls=(WidgetControl(options=['position', 'transparent_bg'],…